# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
%matplotlib inline

In [63]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys1 import geoapify_key

In [64]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,24.24,82,100,6.86,PN,1707170690
1,1,anadyr,64.7500,177.4833,-22.72,77,100,7.00,RU,1707170983
2,2,port-aux-francais,-49.3500,70.2167,5.94,82,99,12.42,TF,1707170983
3,3,whitehorse,60.7161,-135.0538,-11.06,95,100,1.22,CA,1707170983
4,4,vilyuchinsk,52.9306,158.4028,-9.85,76,78,3.78,RU,1707170983


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='City',
    cmap='Category20',
    tiles='OSM',
    frame_width=800,
    frame_height=600
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df[
    (city_data_df['Max Temp'] >= 20 ) &
    (city_data_df['Max Temp'] <= 35 ) & 
    (city_data_df['Humidity'] < 75 ) &
    (city_data_df['Cloudiness'] <= 20 ) &
    (city_data_df['Wind Speed'] < 4)
]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna().reset_index(drop=True)

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,89,cabo san lucas,22.8909,-109.9124,30.90,41,20,3.09,MX,1707170761
1,107,peddapalli,18.6167,79.3667,22.92,64,0,1.56,IN,1707170907
2,155,mahbubnagar,16.7333,77.9833,22.85,58,0,1.30,IN,1707171041
3,170,san luis de la loma,17.2706,-100.8939,28.34,62,2,3.36,MX,1707171046
4,177,brades,16.7918,-62.2106,29.76,65,20,1.03,MS,1707171048
5,268,al ghayzah,16.2079,52.1760,22.85,65,3,1.76,YE,1707171078
6,334,coahuayana de hidalgo,18.7000,-103.6583,30.49,51,5,3.54,MX,1707171111
7,375,comitancillo,15.0833,-91.7167,23.24,44,9,2.21,GT,1707171122
8,406,karratha,-20.7377,116.8463,26.89,68,14,1.61,AU,1707170820
9,412,taltal,-25.4000,-70.4833,23.32,73,0,2.15,CL,1707171134


### Step 3: Create a new DataFrame called `hotel_df`.

In [67]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,cabo san lucas,MX,22.8909,-109.9124,41,
1,peddapalli,IN,18.6167,79.3667,64,
2,mahbubnagar,IN,16.7333,77.9833,58,
3,san luis de la loma,MX,17.2706,-100.8939,62,
4,brades,MS,16.7918,-62.2106,65,
5,al ghayzah,YE,16.2079,52.1760,65,
6,coahuayana de hidalgo,MX,18.7000,-103.6583,51,
7,comitancillo,GT,15.0833,-91.7167,44,
8,karratha,AU,-20.7377,116.8463,68,
9,taltal,CL,-25.4000,-70.4833,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [75]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'type': 'accomodation',
    'limit' : 1,
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"]
    lat = hotel_df.loc[index,"Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: No hotel found
peddapalli - nearest hotel: No hotel found
mahbubnagar - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
brades - nearest hotel: No hotel found
al ghayzah - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
comitancillo - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
taltal - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
raja - nearest hotel: No hotel found
cosala - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,cabo san lucas,MX,22.8909,-109.9124,41,No hotel found
1,peddapalli,IN,18.6167,79.3667,64,No hotel found
2,mahbubnagar,IN,16.7333,77.9833,58,No hotel found
3,san luis de la loma,MX,17.2706,-100.8939,62,No hotel found
4,brades,MS,16.7918,-62.2106,65,No hotel found
5,al ghayzah,YE,16.2079,52.1760,65,No hotel found
6,coahuayana de hidalgo,MX,18.7000,-103.6583,51,No hotel found
7,comitancillo,GT,15.0833,-91.7167,44,No hotel found
8,karratha,AU,-20.7377,116.8463,68,No hotel found
9,taltal,CL,-25.4000,-70.4833,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
name_address 

{'statusCode': 401, 'error': 'Unauthorized', 'message': 'Invalid apiKey'}

In [71]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='City',
    hover_cols=['Hotel Name', 'Country'],
    tiles='OSM'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)